In [1]:
import os
import time
import openai
from tqdm import tqdm
import configparser
import pickle
import queue
import json
from scipy.cluster.hierarchy import dendrogram, linkage

API Stuff

In [2]:
config_path = "../config.ini"
config = configparser.ConfigParser()
config.read(config_path)
openai_api_key = config.get('openai', 'api_key')
openai.api_key = openai_api_key

## Get Embeddings and Summaries

In [3]:
with open("../Data/openai_summaries_2", "rb") as fp:
    summaries = pickle.load(fp)
with open("../Helper Stuff/embeddings.json") as f:
    embeddings = json.load(f)

In [4]:
reverse_embeddings = {}
for i in range(len(embeddings.keys())):
    reverse_embeddings[tuple(list(embeddings.values())[i])] = list(embeddings.keys())[i]

## Creating Labels from Linkage Matrix

Helper Functions

In [5]:
def summary_from_embedding(vector, reverse_embeddings, summaries):
    file = reverse_embeddings[tuple(vector)]
    return [summaries[file]]

def determine_label(topics, model_engine="gpt-3.5-turbo", prompt = "In a minimum of 1 word and a maximum of 3 words find the most specific commonality between the following topics:"):
    main_text = "{} {}".format(prompt, " ".join(topics))
    prompt = {"role":"system", "content": main_text}
    try:
        response = openai.ChatCompletion.create(model = model_engine, messages = [prompt])["choices"][0]["message"]["content"]
    except:
        print("Error with rate limit waiting 60 seconds")
        time.sleep(60)
        response = openai.ChatCompletion.create(model = model_engine, messages = [prompt])["choices"][0]["message"]["content"]
    return response


The fourth index is the label and the fifth index are all the topics (previous labels) used to generate that label

In [6]:
embedding_vals = list(embeddings.values())
links = [list(x) for x in linkage(embedding_vals)]
original_size = len(embedding_vals)

for i in tqdm(range(len(links))):
    cluster = links[i]
    sub_cluster_a = cluster[0]
    sub_cluster_b = cluster[1]
    cluster.append(None)
    if sub_cluster_a < original_size:
        summary1 = summary_from_embedding(embedding_vals[int(sub_cluster_a)], reverse_embeddings, summaries)
    else:
        i = int(sub_cluster_a - original_size)
        summary1 = links[i][5]
    if sub_cluster_b < original_size:
        summary2 = summary_from_embedding(embedding_vals[int(sub_cluster_b)], reverse_embeddings, summaries)
    else:
        i = int(sub_cluster_b - original_size)
        #print(links)
        summary2 = links[i][5]
    cluster.append(summary1 + summary2)
    cluster[4] = determine_label(cluster[5])
    links[i] = cluster
    #print(cluster)

 62%|██████▏   | 365/593 [04:25<03:05,  1.23it/s]

Error with rate limit waiting 60 seconds
Done Sleeping


 91%|█████████ | 541/593 [08:01<00:41,  1.26it/s]  

Error with rate limit waiting 60 seconds
Done Sleeping


100%|█████████▉| 592/593 [10:10<00:00,  1.23it/s]

Error with rate limit waiting 60 seconds
Done Sleeping


100%|██████████| 593/593 [11:11<00:00,  1.13s/it]


Saving File:

In [9]:
path = "../Data/LinkageLabels.pkl"
with open(path, "wb") as fp:
    pickle.dump(links, fp)

How to Load:

In [10]:
with open(path, "rb") as fp:
    loaded_links = pickle.load(fp)

In [11]:
loaded_links

[[399.0,
  594.0,
  0.42786448395325166,
  3.0,
  'Esoteric literature.',
  ['Book - Philosophy - Alchemy', 'Secret Societies.', 'Book, 1880, France.']],
 [204.0,
  595.0,
  0.15685175833221093,
  3.0,
  'Shakespearean drama.',
  ['Category: Theater.',
   "Literature - Shakespeare's play.",
   'Literature/purchase/bargain.']],
 [303.0,
  596.0,
  0.1652177944881287,
  4.0,
  'Shakespearean literature.',
  ['Shakespearean play analysis.',
   'Category: Theater.',
   "Literature - Shakespeare's play.",
   'Literature/purchase/bargain.']],
 [303.0,
  596.0,
  0.1652177944881287,
  4.0,
  'Shakespearean literature.',
  ['Shakespearean play analysis.',
   'Category: Theater.',
   "Literature - Shakespeare's play.",
   'Literature/purchase/bargain.']],
 [277.0,
  379.0,
  0.1695001531610153,
  2.0,
  'Russian fiction.',
  ['Literature, Russian, Free.', 'Book, Fiction, Russian-Language.']],
 [288.0,
  599.0,
  0.4305627017428426,
  3.0,
  'Machines.',
  ['Engineering book.', 'Book: Machines (